In [8]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

model = load_model('models//')

In [ ]:
mixer.init()
sound = mixer.Sound('sound/alarm.wav')

cap = cv2.VideoCapture(0)
score = 0
while True:
    try:
        ret, frame = cap.read()
        height, width = frame.shape[0:2]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
        eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=1)


        cv2.rectangle(frame, (0, height - 50), (200, height), (0,0,0), thickness=cv2.FILLED)

#         Put box for face        
#         for (x, y, w, h) in face:
#             cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 0), thickness=3)

#         Prediction
#         Needs model to run the code below
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(frame, pt1=(ex, ey), pt2=(ex+ew, ey+eh), color=(255, 0, 0), thickness=3)
            # preprocessing
            eye = frame[ey:ey+eh, ex:ex+ew]
            eye = cv2.resize(eye, (80, 80))
            eye = eye/255
            eye = eye.reshape(80, 80, 3)
            eye = np.expand_dims(eye, axis=0)
            
            # model prediction
            prediction = model.predict(eye)
            
#           if eyes are at least 30% closed
            if prediction[0][0] > 0.30:
                cv2.putText(frame, 'Closed', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score : '+str(score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
                
                score += 1
                if score > 15:
                    try:
                        sound.play()
                    except Exception as e:
                        print(e)

#           if eyes are at least 90% opened    
            elif prediction[0][1] > 0.90:
                cv2.putText(frame, 'Opened', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)    
                cv2.putText(frame, 'Score : '+str(score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
                
                score -= 1
                if score < 0:
                    score = 0

                
        cv2.imshow('Frame', frame)

        if cv2.waitKey(33) & 0xFF==ord('q'):
            break
    except Exception as e:
        print(e)
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 37ms/step
